# **Data Cleaning**

Let's first gather the data, and go through the cleaning process. Afterwards, let's calculate the statistics for specific groups.

As you can tell, I first check the percentage of NAN values for each column to see if I should drop or impute the column. Secondly, I checked to see if there are distinct unique values in the remaining columns. "state" and "police_department" all had the same values, so I dropped those columns. Furthermore, I found out that all the raw data columns had corresponding clean data columns, thus I dropped the raw data columns. Also, "contraband_found" had a heavily skewed dataset pointing to False, thus it had very little information for the models I am planning to big. "officer_id" was dropped due to multiple instances of false data inputs.

Finally, I drop all the rows with NAN values in the features where I absolutely need for my dataset.

In [2]:
import pandas as pd

soppFile = '/content/NC-clean.csv'
soppData = pd.read_csv(soppFile)
soppData = soppData.drop("id",axis=1)

#Calculated the percentage of NAN values in each column to see if I should drop or impute
for x in soppData.columns:
  total = soppData[x].isna().sum()
  print("Percentage of NAN for", x, ":", total/len(soppData))

print()
#from the list, you can tell that stop_time, search_type_raw, search_type,
#fine_grained_location, drugs_related_stop,county_name,county_fips
#location_raw and search_basis should be dropped as they have a high amounts of NA
soppData = soppData.drop(['stop_time',
                          'search_type_raw',
                          'search_type',
                          'search_basis',
                          'fine_grained_location',
                          'drugs_related_stop',
                          'county_name',
                          'county_fips',
                          'location_raw'],
                           axis=1)
#Next, let's see if there's a distinct unique values in the remaining columns
for x in soppData.columns:
  total = soppData[x].unique()
  print(x, total)
print()
#we can get rid of all the raw data columns as well as state as it has only one unique value
#Furthermore we can get rid of police_department as both unique values are the same entity
soppData = soppData.drop(['state',
                          'driver_age_raw',
                          'driver_race_raw',
                          'violation_raw',
                          'police_department'],
                          axis = 1)

#After checking the distributions for contraband_found, there is a heavy majority
#thus these can be dropped as well
#officer_id is dropped as there are multiple incorrect inputs such as "TestHW"
soppData = soppData.drop(['contraband_found',"officer_id"], axis=1)

#to calculate is_arrested drop the rows where they are not found for race, is_arrested, and gender
soppData.dropna(subset=['is_arrested', 'driver_race', 'driver_gender'],inplace=True)

soppData



Percentage of NAN for state : 0.0
Percentage of NAN for stop_date : 0.0
Percentage of NAN for stop_time : 0.002563554795983764
Percentage of NAN for location_raw : 0.99954222235786
Percentage of NAN for county_name : 0.99969481490524
Percentage of NAN for county_fips : 0.99969481490524
Percentage of NAN for fine_grained_location : 0.0004577776421399579
Percentage of NAN for police_department : 0.0
Percentage of NAN for driver_gender : 0.0
Percentage of NAN for driver_age_raw : 0.0
Percentage of NAN for driver_age : 0.0004882961516159551
Percentage of NAN for driver_race_raw : 0.0
Percentage of NAN for driver_race : 0.0
Percentage of NAN for violation_raw : 0.0
Percentage of NAN for violation : 0.0
Percentage of NAN for search_conducted : 0.0
Percentage of NAN for search_type_raw : 0.9843134861293374
Percentage of NAN for search_type : 0.9843134861293374
Percentage of NAN for contraband_found : 0.0
Percentage of NAN for stop_outcome : 0.0
Percentage of NAN for is_arrested : 3.0518509475

,stop_date,driver_gender,driver_age,driver_race,violation,search_conducted,stop_outcome,is_arrested,ethnicity,district
0,2000-01-01,M,35.0,Black,Equipment,True,Arrest,True,N,NaN
1,2000-01-01,M,20.0,White,Other,False,Written Warning,False,N,NaN
2,2000-01-01,F,26.0,White,Safe movement,False,Written Warning,False,N,NaN
3,2000-01-01,F,48.0,White,Speeding,False,Citation,False,N,NaN
4,2000-01-01,F,18.0,Black,Speeding,False,No Action,False,N,NaN
...,...,...,...,...,...,...,...,...,...,...
32761,2000-01-20,M,38.0,White,Other,False,Citation,False,N,NaN
32762,2000-01-20,M,28.0,White,Equipment,False,Citation,False,N,NaN
32763,2000-01-20,M,24.0,White,Other,False,Citation,False,N,NaN
32764,2000-01-20,M,37.0,White,Registration/plates,False,Citation,False,N,NaN


In the next code cell, I try to fill out even more NAN values where I utilized .fillna() for my "district" column (Note: this was unnecessary as I thought I would utilize these columns, but in the end, only utilized driver's age, gender, and race.)

Afterwards, I bin the ages within intervals.

In [3]:
#With the remaining columns let's clean it even further by filling out the rest of NAN
for x in soppData.columns:
  total = soppData[x].unique()
  print(x, total)
print()
#only district has NaN values so fill it up with mode()
print(soppData["district"].mode())
soppData["district"] = soppData["district"].fillna("C3")

ageBins = [15,19,29,39,49,200]
ageLabel = ["15-19", "20-29", "30-39","40-49","50+"]

print()
#next let's bin the age ranges
soppData["driver_age"] = pd.cut(soppData["driver_age"],bins=ageBins,labels=ageLabel,right=False)
print(soppData["driver_age"].value_counts())
soppData

stop_date ['2000-01-01' '2000-01-02' '2000-01-03' '2000-01-04' '2000-01-05'
 '2000-01-06' '2000-01-07' '2000-01-08' '2000-01-09' '2000-01-10'
 '2000-01-11' '2000-01-12' '2000-01-13' '2000-01-14' '2000-01-15'
 '2000-01-16' '2000-01-17' '2000-01-18' '2000-01-19' '2000-01-20']
driver_gender ['M' 'F']
driver_age [35. 20. 26. 48. 18. 25. 30. 40. 28. 21. 22. 31. 23. 27. 50. 44. 17. 19.
 60. 33. 45. 37. 49. 24. 54. 34. 41. 63. 29. 67. 32. 16. 57. 36. 46. 86.
 43. 42. 77. 59. 38. 51. 66. 65. 79. 39. 52. 55. 64. 58. 53. 85. 69. 47.
 72. 71. 56. 15. 74. 62. 61. 68. 76. nan 81. 70. 73. 78. 75. 84. 83. 80.
 92. 90. 82. 97. 87. 91.]
driver_race ['Black' 'White' 'Other' 'Hispanic' 'Asian']
violation ['Equipment' 'Other' 'Safe movement' 'Speeding' 'DUI' 'Seat belt'
 'Stop sign/light' 'Registration/plates']
search_conducted [ True False]
stop_outcome ['Arrest' 'Written Warning' 'Citation' 'No Action' 'Verbal Warning']
is_arrested [True False]
ethnicity ['N' 'H']
district [nan 'F4' 'C3' 'E5' 'H6']

0  

,stop_date,driver_gender,driver_age,driver_race,violation,search_conducted,stop_outcome,is_arrested,ethnicity,district
0,2000-01-01,M,30-39,Black,Equipment,True,Arrest,True,N,C3
1,2000-01-01,M,20-29,White,Other,False,Written Warning,False,N,C3
2,2000-01-01,F,20-29,White,Safe movement,False,Written Warning,False,N,C3
3,2000-01-01,F,40-49,White,Speeding,False,Citation,False,N,C3
4,2000-01-01,F,15-19,Black,Speeding,False,No Action,False,N,C3
...,...,...,...,...,...,...,...,...,...,...
32761,2000-01-20,M,30-39,White,Other,False,Citation,False,N,C3
32762,2000-01-20,M,20-29,White,Equipment,False,Citation,False,N,C3
32763,2000-01-20,M,20-29,White,Other,False,Citation,False,N,C3
32764,2000-01-20,M,30-39,White,Registration/plates,False,Citation,False,N,C3


# **Part A**

Let's find the percentage of each possible group defined by race,age,
and gender and if they were arrested or not.

In [4]:
#Calculate Races

#first get the percentage of each race being arrested
print(soppData["driver_race"].value_counts())
print()

asian_arrest = soppData[soppData["driver_race"] == "Asian"]
total_asiansArrest = asian_arrest[asian_arrest["is_arrested"] == True].shape[0]
total = asian_arrest.shape[0]
print("Asians Arrested:", total_asiansArrest/total)

black_arrest = soppData[soppData["driver_race"] == "Black"]
total_black = black_arrest[black_arrest["is_arrested"] == True].shape[0]
total = black_arrest.shape[0]
print("Black Arrested:", total_black/total)

white_arrest = soppData[soppData["driver_race"] == "White"]
total_white = white_arrest[white_arrest["is_arrested"] == True].shape[0]
total = white_arrest.shape[0]
print("White Arrested:", total_white/total)

hisp_arrest = soppData[soppData["driver_race"] == "Hispanic"]
total_hisp = hisp_arrest[hisp_arrest["is_arrested"] == True].shape[0]
total = hisp_arrest.shape[0]
print("Hispanic Arrested:", total_hisp/total)

White       22492
Black        7537
Hispanic     1263
Other        1192
Asian         282
Name: driver_race, dtype: int64

Asians Arrested: 0.010638297872340425
Black Arrested: 0.028127902348414488
White Arrested: 0.019518050862528898
Hispanic Arrested: 0.06175771971496437


In [5]:
#Caclulate per Gender
print(soppData["driver_gender"].value_counts())
print()

f_arrest = soppData[soppData["driver_gender"] == "F"]
total_f = f_arrest[f_arrest["is_arrested"] == True].shape[0]
total = f_arrest.shape[0]
print("Females Arrested:", total_f/total)


m_arrest = soppData[soppData["driver_gender"] == "M"]
total_m = m_arrest[m_arrest["is_arrested"] == True].shape[0]
total = m_arrest.shape[0]
print("Males Arrested:", total_m/total)

M    22647
F    10119
Name: driver_gender, dtype: int64

Females Arrested: 0.011957703330368613
Males Arrested: 0.029893584139179582


In [6]:
#Calculate per Age Bin
print(soppData["driver_age"].value_counts())
print()

bin1 = soppData[soppData["driver_age"] == "15-19"]
total_bin1 = bin1[bin1["is_arrested"] == True].shape[0]
total = bin1.shape[0]
print("15-19 Years Arrested:", total_bin1/total)

bin2 = soppData[soppData["driver_age"] == "20-29"]
total_bin2 = bin2[bin2["is_arrested"] == True].shape[0]
total = bin2.shape[0]
print("20-29 Years Arrested:", total_bin2/total)

bin3 = soppData[soppData["driver_age"] == "30-39"]
total_bin3 = bin3[bin3["is_arrested"] == True].shape[0]
total = bin3.shape[0]
print("30-39 Years Arrested:", total_bin3/total)

bin4 = soppData[soppData["driver_age"] == "40-49"]
total_bin4 = bin4[bin4["is_arrested"] == True].shape[0]
total = bin4.shape[0]
print("40-49 Years Arrested:", total_bin4/total)

bin5 = soppData[soppData["driver_age"] == "50+"]
total_bin5 = bin5[bin5["is_arrested"] == True].shape[0]
total = bin5.shape[0]
print("50+ Years Arrested:", total_bin5/total)

20-29    12512
30-39     8155
40-49     5297
50+       4359
15-19     2427
Name: driver_age, dtype: int64

15-19 Years Arrested: 0.013597033374536464
20-29 Years Arrested: 0.024616368286445013
30-39 Years Arrested: 0.030533415082771305
40-49 Years Arrested: 0.026052482537285256
50+ Years Arrested: 0.01605872906629961


# **Part B**

My method of encoding for the driver's age, gender, and race are the following:

Age: Since age is binned, I can use a simple LabelEncoder() to give each unique value a number key, so my logistic regression can handle the categorical data. However, due to needing to find coefficients, I will use OneHotEncoder() to easily access each coefficients.

Gender: Since there are only two genders in the data column, the LabelEncoder() is fine as it implies True or False whether or not a person is male or female. If it were multiple genders, one-hot encoding is best as it does not imply order and increases fairness within the model. However, similar to the age column, I need to find coefficients, thus I will use OneHotEncoder().

Race: Since there are multiple races in the data column, OneHotEncoder() is a must as if we utilize OrdinalEncoder() or LabelEncoder(), we imply that a races have an "order" which can lead to multiple misintrepretations. This can be extremely unethical, and must be tread on carefully to ensure fairness.

In [7]:
#Let's ENCODE!!!
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

cols = ['driver_age','driver_gender',"driver_race"]

ohe = OneHotEncoder(sparse=False)

for c in cols:
  encoded = ohe.fit_transform(soppData[[c]])
  category = ohe.categories_[0]
  encoded = pd.DataFrame(encoded, columns=[f'driver_{cat}' for cat in category])

  soppData.reset_index(drop=True,inplace=True)
  encoded.reset_index(drop=True,inplace=True)

  soppData = pd.concat([soppData,encoded],axis=1)

soppData = soppData.drop(["driver_race","driver_age","driver_gender"],axis=1)

soppData

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,stop_date,violation,search_conducted,stop_outcome,is_arrested,ethnicity,district,driver_15-19,driver_20-29,driver_30-39,driver_40-49,driver_50+,driver_nan,driver_F,driver_M,driver_Asian,driver_Black,driver_Hispanic,driver_Other,driver_White
0,2000-01-01,Equipment,True,Arrest,True,N,C3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2000-01-01,Other,False,Written Warning,False,N,C3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2000-01-01,Safe movement,False,Written Warning,False,N,C3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2000-01-01,Speeding,False,Citation,False,N,C3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2000-01-01,Speeding,False,No Action,False,N,C3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32761,2000-01-20,Other,False,Citation,False,N,C3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
32762,2000-01-20,Equipment,False,Citation,False,N,C3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
32763,2000-01-20,Other,False,Citation,False,N,C3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
32764,2000-01-20,Registration/plates,False,Citation,False,N,C3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


# **Part C**

In the code cell below, I LabelEncode() all my target variables that I am trying to predict: "stop_outcome", "is_arrested", and "search_conducted". I had to encode "stop_outcome" differently as I was only interested if the stop ended with a "Citation" or not, thus I made it into a boolean column.

Afterwards, I set my training sets and validation sets with the all my features. I only utilized driver's gender, age, and race to my X-features. Furthermore, I made 3 different target variables as I am building 3 different models to map those distinct target variables.

I print out the correlation, but there are no major correlations between the different category columns.

Important Step: I made sure to scale my training and validation X-features sets.

Finally, I build the 3 models, and print out their corresponding coefficients and calculate their mean squared errors.


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#Furthermore we need to encode our target variables
label = LabelEncoder()
soppData['stop_outcome'] = label.fit_transform(soppData['stop_outcome'] == "Citation")
soppData['is_arrested'] = label.fit_transform(soppData['is_arrested'])
soppData['search_conducted'] = label.fit_transform(soppData['search_conducted'])

features = ['driver_15-19',	'driver_20-29',	'driver_30-39',	'driver_40-49', 'driver_50+', 'driver_F', 'driver_M'
            ,'driver_Asian', 'driver_Black', 'driver_Hispanic',	'driver_Other',	'driver_White']

#we can drop all other columns that are not gender,age,or race (besides stop_outcome, is_arrested, and seach_conducted as they are our target variables)
#we really only care about driver gender age and race
X = soppData
search = soppData["search_conducted"]
arrested = soppData["is_arrested"]
stop = soppData["stop_outcome"]
X = X.drop(["stop_date","violation","search_conducted","stop_outcome","is_arrested","ethnicity","district","driver_nan"],axis=1)

trainX, valX, trainSearch, valSearch, trainArrested, valArrested, trainStop, valStop = train_test_split(
                              X, search, arrested,stop, test_size = 0.2, random_state=42)

#Check for Correlation
print(X.corr())
print()

#Scale the X-features
scale = StandardScaler()

trainX = scale.fit_transform(trainX)
valX = scale.transform(valX)

#Model for search
model1 = LogisticRegression()

model1.fit(trainX, trainSearch)

pred = model1.predict(valX)

coefficients = model1.coef_

mse = mean_squared_error(pred,valSearch)
print("Search_Conducted Coefficients and MSE")
print()
for col in range(0,len(features)):
  print(features[col],":", coefficients[0][col])
print("MSE",mse)
print()

#Model for Arrested
model2 = LogisticRegression()

model2.fit(trainX, trainArrested)

pred = model2.predict(valX)

coefficients = model2.coef_

mse = mean_squared_error(pred,valArrested)

print("Is_Arrested Coefficients and MSE")
print()
for col in range(0,len(features)):
  print(features[col],":", coefficients[0][col])
print("MSE",mse)
print()

#Model for Citation
model3 = LogisticRegression()
model3.fit(trainX, trainStop)

pred = model3.predict(valX)

coefficients = model3.coef_

mse = mean_squared_error(pred,valStop)

print("Stop_Outcome Coefficients and MSE")
print()
for col in range(0,len(features)):
  print(features[col],":", coefficients[0][col])
print("MSE",mse)


                 driver_15-19  driver_20-29  driver_30-39  driver_40-49  \
driver_15-19         1.000000     -0.222301     -0.162810     -0.124202   
driver_20-29        -0.222301      1.000000     -0.452434     -0.345145   
driver_30-39        -0.162810     -0.452434      1.000000     -0.252779   
driver_40-49        -0.124202     -0.345145     -0.252779      1.000000   
driver_50+          -0.110794     -0.307885     -0.225490     -0.172018   
driver_F             0.017273     -0.001500      0.016122      0.000744   
driver_M            -0.017273      0.001500     -0.016122     -0.000744   
driver_Asian        -0.011213      0.004295      0.008264      0.005754   
driver_Black        -0.054903     -0.002250      0.025351      0.022172   
driver_Hispanic     -0.022732      0.087030     -0.003792     -0.038403   
driver_Other        -0.009518      0.025775      0.011811     -0.004738   
driver_White         0.065310     -0.045329     -0.027834     -0.003411   

                 driver_

# **Part D**


Coefficient Interpretations/Conclusions:

(Note: the numbers might not be the same as shown above due to how I fitted the model. However, the numbers are similar consistently.)

Ratio of Search: Hispanic/White
*   Hispanic Search Coefficient: 0.17396344
*   White Search Coefficient: -0.16708718
*   Ratio: e^(0.17396344 - -0.16708718) = 1.40779652912

Ratio of Search: Black/White
*   Black Search Coefficient: 0.08575688
*   White Search Coefficient: -0.16708718
*   Ratio: e^(0.08575688 - -0.16708718) = 1.28785875834

Ratio of Arrest: Hispanic/White
*   Hispanic Search Coefficient: 0.17352754
*   White Search Coefficient: -0.14028223
*   Ratio: e^(0.17352754 - -0.14028223) = 1.36999397165

Ratio of Arrest: Black/White
*   Black Search Coefficient: 0.04599118
*   White Search Coefficient: -0.14028223
*   Ratio: e^(0.04599118 - -0.14028223) = 1.204

Citation Diffirence: Hispanic/White
*   Hispanic Citation Coefficient: 0.02795596
*   White Citation Coefficient: 0.0087728
*   Difference: .0087728 - .02795596 = −0.01918316

Citation Difference: Black/White
*   Black Citation Coefficient: -0.02442865
*   White Citation Coefficient: 0.0087728
*   Difference: .0087728 - -0.02442865 = 0.03320145

Age/Gender Coefficients:
Based on the data found, gender does have a significant impact on whether or not the person receives a citation, is arrested, or is searched. Females have a negative coefficient which means they are more unlikely to get searched while men have a positive coefficient which means they are more likely to receive those outcomes.

Age shows a pattern as well. All ages seem to have a relatively low coefficients. However, ages 15-19 and 50+ seem to have the lowest chance of the target variables. It may seem that people deemed as teenagers and elderly are given more leeway as they are usually deemed vulnerable by society.
